# El Pollo Loco locations

In [1]:
import requests
import pandas as pd
import geopandas as gpd
from vega_datasets import data
import altair as alt

In [2]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000

In [3]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

In [4]:
headers = {
    "authority": "www.elpolloloco.com",
    "accept": "application/json, text/javascript, */*; q=0.01",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
}

response = requests.get(
    "https://www.elpolloloco.com/locations/locations_json", headers=headers
)

In [5]:
src = pd.DataFrame(
    response.json(),
    columns=[
        "location_no",
        "street_address",
        "blank",
        "city",
        "state",
        "zip",
        "phone",
        "unk",
        "lat",
        "lon",
        "updated",
        "ukn",
        "slug",
        "hours",
        "blank",
        "time_zone",
        "ukn",
        "ukn",
        "closed_status",
        "ukn",
        "url",
        "id",
    ],
).sort_values("updated")

In [6]:
df = src[
    [
        "street_address",
        "city",
        "state",
        "zip",
        "phone",
        "lat",
        "lon",
        "updated",
        "slug",
        "time_zone",
    ]
].copy()

In [7]:
df["updated"] = pd.to_datetime(df["updated"]).dt.strftime("%Y-%m-%d")

In [ ]:
df.head()

In [11]:
len(df)

500

In [9]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lat, df.lon))

In [10]:
states = alt.topo_feature(data.us_10m.url, feature="states")

places = (
    alt.Chart(gdf)
    .mark_circle(color="#ff9900")
    .encode(
        longitude="lon:Q",
        latitude="lat:Q",
        size=alt.value(8),
        tooltip=["street_address", "city", "zip"],
    )
    .project("albersUsa")
    .properties(width=900, height=500, title="El Pollo Loco locations")
)

states = (
    alt.Chart(states).mark_geoshape(fill="#E9E9E9", stroke="white").project("albersUsa")
)

(states + places).configure_view(strokeWidth=0)

alt.LayerChart(...)